# Metadata Generation

This notebook illustrates the process of generating metadata stored in `data` object in the Snakefile.
Each cell adds one column to `data` following the instructions and conditions

In [1]:
import os
import  pandas as pd
import numpy as np
import glob
from snakemake.io import expand
import re

In [2]:
data = pd.read_csv("../config/table_siNipbl.tsv", sep="\t")
data_siSA = pd.read_csv("../config/table_siSA1_siSA2.tsv", sep="\t")
data_siSA

,Run,GEO_Accession (exp),Cell_Line,TREATMENT,Antibody,BioProject,Comments,Protein,Condition,Rep,File,Genome,Norm,Ext
0,SRR6501087,GSM2942285,MCF10A,Control,custom made antibody for SA1 (STAG1),PRJNA395931,NaN,SA1,siC,R1,SA1_siC_R1_SRR6501087.fastq.gz,hg19,mm9,.fastq.gz
1,SRR6501088,GSM2942286,MCF10A,Control,custom made antibody for SA1 (STAG1),PRJNA395931,NaN,SA1,siC,R2,SA1_siC_R2_SRR6501088.fastq.gz,hg19,mm9,.fastq.gz
2,SRR6501089,GSM2942287,MCF10A,Control,custom made antibody for SA2 (STAG2),PRJNA395931,NaN,SA2,siC,R1,SA2_siC_R1_SRR6501089.fastq.gz,hg19,mm9,.fastq.gz
3,SRR6501090,GSM2942288,MCF10A,Control,custom made antibody for SA2 (STAG2),PRJNA395931,NaN,SA2,siC,R2,SA2_siC_R2_SRR6501090.fastq.gz,hg19,mm9,.fastq.gz
4,SRR6501091,GSM2942289,MCF10A,STAG1 siRNAs,custom made antibody for SA1 (STAG1),PRJNA395931,NaN,SA1,siSA1,R1,SA1_siSA1_R1_SRR6501091.fastq.gz,hg19,mm9,.fastq.gz
5,SRR6501092,GSM2942290,MCF10A,STAG1 siRNAs,custom made antibody for SA1 (STAG1),PRJNA395931,NaN,SA1,siSA1,R2,SA1_siSA1_R2_SRR6501092.fastq.gz,hg19,mm9,.fastq.gz
6,SRR6501093,GSM2942291,MCF10A,STAG1 siRNAs,custom made antibody for SA2 (STAG2),PRJNA395931,NaN,SA2,siSA1,R1,SA2_siSA1_R1_SRR6501093.fastq.gz,hg19,mm9,.fastq.gz
7,SRR6501094,GSM2942292,MCF10A,STAG1 siRNAs,custom made antibody for SA2 (STAG2),PRJNA395931,NaN,SA2,siSA1,R2,SA2_siSA1_R2_SRR6501094.fastq.gz,hg19,mm9,.fastq.gz
8,SRR6501095,GSM2942293,MCF10A,STAG2 siRNAs,custom made antibody for SA1 (STAG1),PRJNA395931,NaN,SA1,siSA2,R1,SA1_siSA2_R1_SRR6501095.fastq.gz,hg19,mm9,.fastq.gz
9,SRR6501096,GSM2942294,MCF10A,STAG2 siRNAs,custom made antibody for SA1 (STAG1),PRJNA395931,NaN,SA1,siSA2,R2,SA1_siSA2_R2_SRR6501096.fastq.gz,hg19,mm9,.fastq.gz


In [3]:
## Genome bowtie2 index prefixes paths
genome_path = {
	"mm9":"/storage/scratch01/users/dgimenezl/genomes/mouse/mm9/mm9",
    "mm10":"/storage/scratch01/users/dgimenezl/genomes/mouse/mm10/mm10" ,
    "hg19":"/storage/scratch01/users/dgimenezl/genomes/human/hg19/hg19",
    "hg38":"/storage/scratch01/users/dgimenezl/genomes/human/hg38/hg38",
    "-":""}
refSeq_genes_path = {
	"mm9" : "",
	"mm10" : "",
	"hg19" : "/storage/scratch01/users/aquevedo/genomes/human/hg19/hg19_RefSeqCuratedGenes.bed",
	"hg38" : ""
}
## Genome sizes for big wig computation
genome_size={"mm9":2620345972,
    "mm10":2652783500,
    "hg19":2864785220,
    "hg38":2913022398}

In [4]:
## Add extra cols for salecting the appropriate wildcards path to files
data["Samples"] = data.Protein +"_"+data.Condition+"_"+ data.Rep 
data_siSA["Samples"] = data_siSA.Protein +"_"+data_siSA.Condition+"_"+ data_siSA.Rep 
data.head()

,Protein,Condition,Rep,Ext,Run,File,Genome,Norm,Input,Samples
0,input,siC,S9,L001_R1_001.fastq.gz,NaN,input-SiC_S9_L001_R1_001.fastq.gz,hg19,mm9,NaN,input_siC_S9
1,input,siC,S9,L002_R1_001.fastq.gz,NaN,input-SiC_S9_L002_R1_001.fastq.gz,hg19,mm9,NaN,input_siC_S9
2,input,siC,S9,L003_R1_001.fastq.gz,NaN,input-SiC_S9_L003_R1_001.fastq.gz,hg19,mm9,NaN,input_siC_S9
3,input,siC,S9,L004_R1_001.fastq.gz,NaN,input-SiC_S9_L004_R1_001.fastq.gz,hg19,mm9,NaN,input_siC_S9
4,input,siNipbl,S10,L001_R1_001.fastq.gz,NaN,input-SiNipbl_S10_L001_R1_001.fastq.gz,hg19,mm9,NaN,input_siNipbl_S10


In [5]:
data["Input"] = [ data.Samples[(data.Protein=="input") & (data.Condition==Cond)].values[0] \
                 if Prot != "input" \
                 else "" \
                 for Prot,Cond in zip(data.Protein,data.Condition)  ]
data_siSA["Input"] = [ data_siSA.Samples[(data_siSA.Protein=="input") & (data_siSA.Condition==Cond)].values[0] \
                 if Prot != "input" \
                 else "" \
                 for Prot,Cond in zip(data_siSA.Protein,data_siSA.Condition)  ]
data_siSA

,Run,GEO_Accession (exp),Cell_Line,TREATMENT,Antibody,BioProject,Comments,Protein,Condition,Rep,File,Genome,Norm,Ext,Samples,Input
0,SRR6501087,GSM2942285,MCF10A,Control,custom made antibody for SA1 (STAG1),PRJNA395931,NaN,SA1,siC,R1,SA1_siC_R1_SRR6501087.fastq.gz,hg19,mm9,.fastq.gz,SA1_siC_R1,input_siC_R1
1,SRR6501088,GSM2942286,MCF10A,Control,custom made antibody for SA1 (STAG1),PRJNA395931,NaN,SA1,siC,R2,SA1_siC_R2_SRR6501088.fastq.gz,hg19,mm9,.fastq.gz,SA1_siC_R2,input_siC_R1
2,SRR6501089,GSM2942287,MCF10A,Control,custom made antibody for SA2 (STAG2),PRJNA395931,NaN,SA2,siC,R1,SA2_siC_R1_SRR6501089.fastq.gz,hg19,mm9,.fastq.gz,SA2_siC_R1,input_siC_R1
3,SRR6501090,GSM2942288,MCF10A,Control,custom made antibody for SA2 (STAG2),PRJNA395931,NaN,SA2,siC,R2,SA2_siC_R2_SRR6501090.fastq.gz,hg19,mm9,.fastq.gz,SA2_siC_R2,input_siC_R1
4,SRR6501091,GSM2942289,MCF10A,STAG1 siRNAs,custom made antibody for SA1 (STAG1),PRJNA395931,NaN,SA1,siSA1,R1,SA1_siSA1_R1_SRR6501091.fastq.gz,hg19,mm9,.fastq.gz,SA1_siSA1_R1,input_siSA1_R1
5,SRR6501092,GSM2942290,MCF10A,STAG1 siRNAs,custom made antibody for SA1 (STAG1),PRJNA395931,NaN,SA1,siSA1,R2,SA1_siSA1_R2_SRR6501092.fastq.gz,hg19,mm9,.fastq.gz,SA1_siSA1_R2,input_siSA1_R1
6,SRR6501093,GSM2942291,MCF10A,STAG1 siRNAs,custom made antibody for SA2 (STAG2),PRJNA395931,NaN,SA2,siSA1,R1,SA2_siSA1_R1_SRR6501093.fastq.gz,hg19,mm9,.fastq.gz,SA2_siSA1_R1,input_siSA1_R1
7,SRR6501094,GSM2942292,MCF10A,STAG1 siRNAs,custom made antibody for SA2 (STAG2),PRJNA395931,NaN,SA2,siSA1,R2,SA2_siSA1_R2_SRR6501094.fastq.gz,hg19,mm9,.fastq.gz,SA2_siSA1_R2,input_siSA1_R1
8,SRR6501095,GSM2942293,MCF10A,STAG2 siRNAs,custom made antibody for SA1 (STAG1),PRJNA395931,NaN,SA1,siSA2,R1,SA1_siSA2_R1_SRR6501095.fastq.gz,hg19,mm9,.fastq.gz,SA1_siSA2_R1,input_siSA2_R1
9,SRR6501096,GSM2942294,MCF10A,STAG2 siRNAs,custom made antibody for SA1 (STAG1),PRJNA395931,NaN,SA1,siSA2,R2,SA1_siSA2_R2_SRR6501096.fastq.gz,hg19,mm9,.fastq.gz,SA1_siSA2_R2,input_siSA2_R1


In [6]:
data_siSA.Protein == 'input'
data_siSA.Samples[data_siSA.Protein == 'input']

12    input_siSA2_R1
13    input_siSA2_R2
14    input_siSA1_R1
15    input_siSA1_R2
16      input_siC_R1
17      input_siC_R2
Name: Samples, dtype: object

In [7]:
# inpuit Merged in siSA1 siSA2
data["InputMerged"] = [re.sub("_S[0-9]+$","", ip) if ip != ""
                      else "" for ip in data.Input]
print(data["InputMerged"])


0                  
1                  
2                  
3                  
4                  
5                  
6                  
7                  
8         input_siC
9         input_siC
10        input_siC
11        input_siC
12        input_siC
13        input_siC
14        input_siC
15        input_siC
16        input_siC
17        input_siC
18        input_siC
19        input_siC
20        input_siC
21        input_siC
22        input_siC
23        input_siC
24        input_siC
25        input_siC
26        input_siC
27        input_siC
28        input_siC
29        input_siC
30        input_siC
31        input_siC
32    input_siNipbl
33    input_siNipbl
34    input_siNipbl
35    input_siNipbl
36    input_siNipbl
37    input_siNipbl
38    input_siNipbl
39    input_siNipbl
40    input_siNipbl
41    input_siNipbl
42    input_siNipbl
43    input_siNipbl
44    input_siNipbl
45    input_siNipbl
46    input_siNipbl
47    input_siNipbl
Name: InputMerged, dtype: object


In [8]:
[(ip,ix) for (ip,ix) in enumerate(data_siSA.Input)]

[(0, 'input_siC_R1'),
 (1, 'input_siC_R1'),
 (2, 'input_siC_R1'),
 (3, 'input_siC_R1'),
 (4, 'input_siSA1_R1'),
 (5, 'input_siSA1_R1'),
 (6, 'input_siSA1_R1'),
 (7, 'input_siSA1_R1'),
 (8, 'input_siSA2_R1'),
 (9, 'input_siSA2_R1'),
 (10, 'input_siSA2_R1'),
 (11, 'input_siSA2_R1'),
 (12, ''),
 (13, ''),
 (14, ''),
 (15, ''),
 (16, ''),
 (17, '')]

In [9]:
data_siSA["InputMerged"] = [re.sub("_[SR][0-9]+$","", ip) if ip != ""
                       else "" for ip in data_siSA.Input]
data_siSA["InputMerged"]



0       input_siC
1       input_siC
2       input_siC
3       input_siC
4     input_siSA1
5     input_siSA1
6     input_siSA1
7     input_siSA1
8     input_siSA2
9     input_siSA2
10    input_siSA2
11    input_siSA2
12               
13               
14               
15               
16               
17               
Name: InputMerged, dtype: object

In [10]:
## Otra forma de rellenar input Merged, para no depender de re.sub con un formato fijo en el nombre de Rep
['input_' + data_siSA.Condition[ix] if ip != ''
else ""
for ix,ip in enumerate(data_siSA.Input)]

['input_siC',
 'input_siC',
 'input_siC',
 'input_siC',
 'input_siSA1',
 'input_siSA1',
 'input_siSA1',
 'input_siSA1',
 'input_siSA2',
 'input_siSA2',
 'input_siSA2',
 'input_siSA2',
 '',
 '',
 '',
 '',
 '',
 '']

In [32]:
data["PATH_genome"] = [genome_path[i] for i in data.Genome] 
data[10:15]

,Protein,Condition,Rep,Ext,Run,File,Genome,Norm,Input,Samples,InputMerged,PATH_genome
10,NiPBL,siC,S4,L003_R1_001.fastq.gz,NaN,Sic-NiPBL_S4_L003_R1_001.fastq.gz,hg19,mm9,input_siC_S9,NiPBL_siC_S4,input_siC,/storage/scratch01/users/dgimenezl/genomes/hum...
11,NiPBL,siC,S4,L004_R1_001.fastq.gz,NaN,Sic-NiPBL_S4_L004_R1_001.fastq.gz,hg19,mm9,input_siC_S9,NiPBL_siC_S4,input_siC,/storage/scratch01/users/dgimenezl/genomes/hum...
12,SA1,siC,S1,L001_R1_001.fastq.gz,NaN,Sic-SA1_S1_L001_R1_001.fastq.gz,hg19,mm9,input_siC_S9,SA1_siC_S1,input_siC,/storage/scratch01/users/dgimenezl/genomes/hum...
13,SA1,siC,S1,L002_R1_001.fastq.gz,NaN,Sic-SA1_S1_L002_R1_001.fastq.gz,hg19,mm9,input_siC_S9,SA1_siC_S1,input_siC,/storage/scratch01/users/dgimenezl/genomes/hum...
14,SA1,siC,S1,L003_R1_001.fastq.gz,NaN,Sic-SA1_S1_L003_R1_001.fastq.gz,hg19,mm9,input_siC_S9,SA1_siC_S1,input_siC,/storage/scratch01/users/dgimenezl/genomes/hum...


In [33]:
data["Genome_size"] = [genome_size[i] for i in data.Genome]
data[1:12:2].Condition

1         siC
3         siC
5     siNipbl
7     siNipbl
9         siC
11        siC
Name: Condition, dtype: object

In [34]:
data["PATH_genome_cal"] = [genome_path[i] for i in data.Norm]
data.tail()

,Protein,Condition,Rep,Ext,Run,File,Genome,Norm,Input,Samples,InputMerged,PATH_genome,Genome_size,PATH_genome_cal
43,SA2,siNipbl,S6,L004_R1_001.fastq.gz,NaN,siNipbl-SA2_S6_L004_R1_001.fastq.gz,hg19,mm9,input_siNipbl_S10,SA2_siNipbl_S6,input_siNipbl,/storage/scratch01/users/dgimenezl/genomes/hum...,2864785220,/storage/scratch01/users/dgimenezl/genomes/mou...
44,SMC1,siNipbl,S7,L001_R1_001.fastq.gz,NaN,siNipbl-SMC1_S7_L001_R1_001.fastq.gz,hg19,mm9,input_siNipbl_S10,SMC1_siNipbl_S7,input_siNipbl,/storage/scratch01/users/dgimenezl/genomes/hum...,2864785220,/storage/scratch01/users/dgimenezl/genomes/mou...
45,SMC1,siNipbl,S7,L002_R1_001.fastq.gz,NaN,siNipbl-SMC1_S7_L002_R1_001.fastq.gz,hg19,mm9,input_siNipbl_S10,SMC1_siNipbl_S7,input_siNipbl,/storage/scratch01/users/dgimenezl/genomes/hum...,2864785220,/storage/scratch01/users/dgimenezl/genomes/mou...
46,SMC1,siNipbl,S7,L003_R1_001.fastq.gz,NaN,siNipbl-SMC1_S7_L003_R1_001.fastq.gz,hg19,mm9,input_siNipbl_S10,SMC1_siNipbl_S7,input_siNipbl,/storage/scratch01/users/dgimenezl/genomes/hum...,2864785220,/storage/scratch01/users/dgimenezl/genomes/mou...
47,SMC1,siNipbl,S7,L004_R1_001.fastq.gz,NaN,siNipbl-SMC1_S7_L004_R1_001.fastq.gz,hg19,mm9,input_siNipbl_S10,SMC1_siNipbl_S7,input_siNipbl,/storage/scratch01/users/dgimenezl/genomes/hum...,2864785220,/storage/scratch01/users/dgimenezl/genomes/mou...


In [35]:
data["PATH_refSeq_genes"] = [refSeq_genes_path[i] for i in data.Genome]
data.tail()

,Protein,Condition,Rep,Ext,Run,File,Genome,Norm,Input,Samples,InputMerged,PATH_genome,Genome_size,PATH_genome_cal,PATH_refSeq_genes
43,SA2,siNipbl,S6,L004_R1_001.fastq.gz,NaN,siNipbl-SA2_S6_L004_R1_001.fastq.gz,hg19,mm9,input_siNipbl_S10,SA2_siNipbl_S6,input_siNipbl,/storage/scratch01/users/dgimenezl/genomes/hum...,2864785220,/storage/scratch01/users/dgimenezl/genomes/mou...,/storage/scratch01/users/aquevedo/genomes/huma...
44,SMC1,siNipbl,S7,L001_R1_001.fastq.gz,NaN,siNipbl-SMC1_S7_L001_R1_001.fastq.gz,hg19,mm9,input_siNipbl_S10,SMC1_siNipbl_S7,input_siNipbl,/storage/scratch01/users/dgimenezl/genomes/hum...,2864785220,/storage/scratch01/users/dgimenezl/genomes/mou...,/storage/scratch01/users/aquevedo/genomes/huma...
45,SMC1,siNipbl,S7,L002_R1_001.fastq.gz,NaN,siNipbl-SMC1_S7_L002_R1_001.fastq.gz,hg19,mm9,input_siNipbl_S10,SMC1_siNipbl_S7,input_siNipbl,/storage/scratch01/users/dgimenezl/genomes/hum...,2864785220,/storage/scratch01/users/dgimenezl/genomes/mou...,/storage/scratch01/users/aquevedo/genomes/huma...
46,SMC1,siNipbl,S7,L003_R1_001.fastq.gz,NaN,siNipbl-SMC1_S7_L003_R1_001.fastq.gz,hg19,mm9,input_siNipbl_S10,SMC1_siNipbl_S7,input_siNipbl,/storage/scratch01/users/dgimenezl/genomes/hum...,2864785220,/storage/scratch01/users/dgimenezl/genomes/mou...,/storage/scratch01/users/aquevedo/genomes/huma...
47,SMC1,siNipbl,S7,L004_R1_001.fastq.gz,NaN,siNipbl-SMC1_S7_L004_R1_001.fastq.gz,hg19,mm9,input_siNipbl_S10,SMC1_siNipbl_S7,input_siNipbl,/storage/scratch01/users/dgimenezl/genomes/hum...,2864785220,/storage/scratch01/users/dgimenezl/genomes/mou...,/storage/scratch01/users/aquevedo/genomes/huma...


In [36]:
## Remove .fastq.gz to use basename with expand() in rule "all"
data["fqBasename"] = [f.replace(".fastq.gz","") for f in data["File"]]
data.tail()

,Protein,Condition,Rep,Ext,Run,File,Genome,Norm,Input,Samples,InputMerged,PATH_genome,Genome_size,PATH_genome_cal,PATH_refSeq_genes,fqBasename
43,SA2,siNipbl,S6,L004_R1_001.fastq.gz,NaN,siNipbl-SA2_S6_L004_R1_001.fastq.gz,hg19,mm9,input_siNipbl_S10,SA2_siNipbl_S6,input_siNipbl,/storage/scratch01/users/dgimenezl/genomes/hum...,2864785220,/storage/scratch01/users/dgimenezl/genomes/mou...,/storage/scratch01/users/aquevedo/genomes/huma...,siNipbl-SA2_S6_L004_R1_001
44,SMC1,siNipbl,S7,L001_R1_001.fastq.gz,NaN,siNipbl-SMC1_S7_L001_R1_001.fastq.gz,hg19,mm9,input_siNipbl_S10,SMC1_siNipbl_S7,input_siNipbl,/storage/scratch01/users/dgimenezl/genomes/hum...,2864785220,/storage/scratch01/users/dgimenezl/genomes/mou...,/storage/scratch01/users/aquevedo/genomes/huma...,siNipbl-SMC1_S7_L001_R1_001
45,SMC1,siNipbl,S7,L002_R1_001.fastq.gz,NaN,siNipbl-SMC1_S7_L002_R1_001.fastq.gz,hg19,mm9,input_siNipbl_S10,SMC1_siNipbl_S7,input_siNipbl,/storage/scratch01/users/dgimenezl/genomes/hum...,2864785220,/storage/scratch01/users/dgimenezl/genomes/mou...,/storage/scratch01/users/aquevedo/genomes/huma...,siNipbl-SMC1_S7_L002_R1_001
46,SMC1,siNipbl,S7,L003_R1_001.fastq.gz,NaN,siNipbl-SMC1_S7_L003_R1_001.fastq.gz,hg19,mm9,input_siNipbl_S10,SMC1_siNipbl_S7,input_siNipbl,/storage/scratch01/users/dgimenezl/genomes/hum...,2864785220,/storage/scratch01/users/dgimenezl/genomes/mou...,/storage/scratch01/users/aquevedo/genomes/huma...,siNipbl-SMC1_S7_L003_R1_001
47,SMC1,siNipbl,S7,L004_R1_001.fastq.gz,NaN,siNipbl-SMC1_S7_L004_R1_001.fastq.gz,hg19,mm9,input_siNipbl_S10,SMC1_siNipbl_S7,input_siNipbl,/storage/scratch01/users/dgimenezl/genomes/hum...,2864785220,/storage/scratch01/users/dgimenezl/genomes/mou...,/storage/scratch01/users/aquevedo/genomes/huma...,siNipbl-SMC1_S7_L004_R1_001


In [37]:
## Column needed to match properly different replicates.
## Each entry is a string of (Protein, Condition) joined by "_"
data["Prot_Cond"] = ["_".join((Prot,Cond)) for Prot,Cond in zip(data.Protein,data.Condition)]

data_siSA["Prot_Cond"] = ["_".join((Prot,Cond)) for Prot,Cond in zip(data_siSA.Protein,data_siSA.Condition)]
data_siSA

,Run,GEO_Accession (exp),Cell_Line,TREATMENT,Antibody,BioProject,Comments,Protein,Condition,Rep,File,Genome,Norm,Ext,Samples,Input,InputMerged,Prot_Cond
0,SRR6501087,GSM2942285,MCF10A,Control,custom made antibody for SA1 (STAG1),PRJNA395931,NaN,SA1,siC,R1,SA1_siC_R1_SRR6501087.fastq.gz,hg19,mm9,.fastq.gz,SA1_siC_R1,input_siC_R1,input_siC,SA1_siC
1,SRR6501088,GSM2942286,MCF10A,Control,custom made antibody for SA1 (STAG1),PRJNA395931,NaN,SA1,siC,R2,SA1_siC_R2_SRR6501088.fastq.gz,hg19,mm9,.fastq.gz,SA1_siC_R2,input_siC_R1,input_siC,SA1_siC
2,SRR6501089,GSM2942287,MCF10A,Control,custom made antibody for SA2 (STAG2),PRJNA395931,NaN,SA2,siC,R1,SA2_siC_R1_SRR6501089.fastq.gz,hg19,mm9,.fastq.gz,SA2_siC_R1,input_siC_R1,input_siC,SA2_siC
3,SRR6501090,GSM2942288,MCF10A,Control,custom made antibody for SA2 (STAG2),PRJNA395931,NaN,SA2,siC,R2,SA2_siC_R2_SRR6501090.fastq.gz,hg19,mm9,.fastq.gz,SA2_siC_R2,input_siC_R1,input_siC,SA2_siC
4,SRR6501091,GSM2942289,MCF10A,STAG1 siRNAs,custom made antibody for SA1 (STAG1),PRJNA395931,NaN,SA1,siSA1,R1,SA1_siSA1_R1_SRR6501091.fastq.gz,hg19,mm9,.fastq.gz,SA1_siSA1_R1,input_siSA1_R1,input_siSA1,SA1_siSA1
5,SRR6501092,GSM2942290,MCF10A,STAG1 siRNAs,custom made antibody for SA1 (STAG1),PRJNA395931,NaN,SA1,siSA1,R2,SA1_siSA1_R2_SRR6501092.fastq.gz,hg19,mm9,.fastq.gz,SA1_siSA1_R2,input_siSA1_R1,input_siSA1,SA1_siSA1
6,SRR6501093,GSM2942291,MCF10A,STAG1 siRNAs,custom made antibody for SA2 (STAG2),PRJNA395931,NaN,SA2,siSA1,R1,SA2_siSA1_R1_SRR6501093.fastq.gz,hg19,mm9,.fastq.gz,SA2_siSA1_R1,input_siSA1_R1,input_siSA1,SA2_siSA1
7,SRR6501094,GSM2942292,MCF10A,STAG1 siRNAs,custom made antibody for SA2 (STAG2),PRJNA395931,NaN,SA2,siSA1,R2,SA2_siSA1_R2_SRR6501094.fastq.gz,hg19,mm9,.fastq.gz,SA2_siSA1_R2,input_siSA1_R1,input_siSA1,SA2_siSA1
8,SRR6501095,GSM2942293,MCF10A,STAG2 siRNAs,custom made antibody for SA1 (STAG1),PRJNA395931,NaN,SA1,siSA2,R1,SA1_siSA2_R1_SRR6501095.fastq.gz,hg19,mm9,.fastq.gz,SA1_siSA2_R1,input_siSA2_R1,input_siSA2,SA1_siSA2
9,SRR6501096,GSM2942294,MCF10A,STAG2 siRNAs,custom made antibody for SA1 (STAG1),PRJNA395931,NaN,SA1,siSA2,R2,SA1_siSA2_R2_SRR6501096.fastq.gz,hg19,mm9,.fastq.gz,SA1_siSA2_R2,input_siSA2_R1,input_siSA2,SA1_siSA2


In [38]:
DATADIR = '/storage/scratch01/users/aquevedo/6Nov-siNipbl-newSnak/data/'
## Check that expand function works as expected
fm= expand(DATADIR + "align/{Prot_Cond}_final_merged.bam",
            Prot_Cond=data.Prot_Cond[data.Protein != 'input'].unique())


In [39]:
RESDIR = '/storage/scratch01/users/aquevedo/6Nov-siNipbl-newSnak/res/'
print("\n".join(expand(RESDIR + 'macs/{Prot_Cond}_merged_predictd.txt',
				Prot_Cond=data.Prot_Cond[data.Protein!="input"].unique())))



/storage/scratch01/users/aquevedo/6Nov-siNipbl-newSnak/res/macs/NiPBL_siC_merged_predictd.txt
/storage/scratch01/users/aquevedo/6Nov-siNipbl-newSnak/res/macs/SA1_siC_merged_predictd.txt
/storage/scratch01/users/aquevedo/6Nov-siNipbl-newSnak/res/macs/SA2_siC_merged_predictd.txt
/storage/scratch01/users/aquevedo/6Nov-siNipbl-newSnak/res/macs/SMC1_siC_merged_predictd.txt
/storage/scratch01/users/aquevedo/6Nov-siNipbl-newSnak/res/macs/NiPBL_siNipbl_merged_predictd.txt
/storage/scratch01/users/aquevedo/6Nov-siNipbl-newSnak/res/macs/SA1_siNipbl_merged_predictd.txt
/storage/scratch01/users/aquevedo/6Nov-siNipbl-newSnak/res/macs/SA2_siNipbl_merged_predictd.txt
/storage/scratch01/users/aquevedo/6Nov-siNipbl-newSnak/res/macs/SMC1_siNipbl_merged_predictd.txt


In [40]:
print("\n".join(expand(RESDIR + 'macs/{Prot_Cond}_merged_predictd.txt',
				Prot_Cond=data_siSA.Prot_Cond[data_siSA.Protein!="input"].unique())))

/storage/scratch01/users/aquevedo/6Nov-siNipbl-newSnak/res/macs/SA1_siC_merged_predictd.txt
/storage/scratch01/users/aquevedo/6Nov-siNipbl-newSnak/res/macs/SA2_siC_merged_predictd.txt
/storage/scratch01/users/aquevedo/6Nov-siNipbl-newSnak/res/macs/SA1_siSA1_merged_predictd.txt
/storage/scratch01/users/aquevedo/6Nov-siNipbl-newSnak/res/macs/SA2_siSA1_merged_predictd.txt
/storage/scratch01/users/aquevedo/6Nov-siNipbl-newSnak/res/macs/SA1_siSA2_merged_predictd.txt
/storage/scratch01/users/aquevedo/6Nov-siNipbl-newSnak/res/macs/SA2_siSA2_merged_predictd.txt


In [41]:
## Column to indicate if need to merge bigWig
## For a given protein and condition, get the unique values of rep. If more than 1, merge

data["MergeBW"]=["yes" if len(data.Rep[(data.Condition==c) & (data.Protein==p)].unique()) > 1
 else "no"
 for c,p in zip(data.Condition, data.Protein)]

data

,Protein,Condition,Rep,Ext,Run,File,Genome,Norm,Input,Samples,InputMerged,PATH_genome,Genome_size,PATH_genome_cal,PATH_refSeq_genes,fqBasename,Prot_Cond,MergeBW
0,input,siC,S9,L001_R1_001.fastq.gz,NaN,input-SiC_S9_L001_R1_001.fastq.gz,hg19,mm9,,input_siC_S9,,/storage/scratch01/users/dgimenezl/genomes/hum...,2864785220,/storage/scratch01/users/dgimenezl/genomes/mou...,/storage/scratch01/users/aquevedo/genomes/huma...,input-SiC_S9_L001_R1_001,input_siC,no
1,input,siC,S9,L002_R1_001.fastq.gz,NaN,input-SiC_S9_L002_R1_001.fastq.gz,hg19,mm9,,input_siC_S9,,/storage/scratch01/users/dgimenezl/genomes/hum...,2864785220,/storage/scratch01/users/dgimenezl/genomes/mou...,/storage/scratch01/users/aquevedo/genomes/huma...,input-SiC_S9_L002_R1_001,input_siC,no
2,input,siC,S9,L003_R1_001.fastq.gz,NaN,input-SiC_S9_L003_R1_001.fastq.gz,hg19,mm9,,input_siC_S9,,/storage/scratch01/users/dgimenezl/genomes/hum...,2864785220,/storage/scratch01/users/dgimenezl/genomes/mou...,/storage/scratch01/users/aquevedo/genomes/huma...,input-SiC_S9_L003_R1_001,input_siC,no
3,input,siC,S9,L004_R1_001.fastq.gz,NaN,input-SiC_S9_L004_R1_001.fastq.gz,hg19,mm9,,input_siC_S9,,/storage/scratch01/users/dgimenezl/genomes/hum...,2864785220,/storage/scratch01/users/dgimenezl/genomes/mou...,/storage/scratch01/users/aquevedo/genomes/huma...,input-SiC_S9_L004_R1_001,input_siC,no
4,input,siNipbl,S10,L001_R1_001.fastq.gz,NaN,input-SiNipbl_S10_L001_R1_001.fastq.gz,hg19,mm9,,input_siNipbl_S10,,/storage/scratch01/users/dgimenezl/genomes/hum...,2864785220,/storage/scratch01/users/dgimenezl/genomes/mou...,/storage/scratch01/users/aquevedo/genomes/huma...,input-SiNipbl_S10_L001_R1_001,input_siNipbl,no
5,input,siNipbl,S10,L002_R1_001.fastq.gz,NaN,input-SiNipbl_S10_L002_R1_001.fastq.gz,hg19,mm9,,input_siNipbl_S10,,/storage/scratch01/users/dgimenezl/genomes/hum...,2864785220,/storage/scratch01/users/dgimenezl/genomes/mou...,/storage/scratch01/users/aquevedo/genomes/huma...,input-SiNipbl_S10_L002_R1_001,input_siNipbl,no
6,input,siNipbl,S10,L003_R1_001.fastq.gz,NaN,input-SiNipbl_S10_L003_R1_001.fastq.gz,hg19,mm9,,input_siNipbl_S10,,/storage/scratch01/users/dgimenezl/genomes/hum...,2864785220,/storage/scratch01/users/dgimenezl/genomes/mou...,/storage/scratch01/users/aquevedo/genomes/huma...,input-SiNipbl_S10_L003_R1_001,input_siNipbl,no
7,input,siNipbl,S10,L004_R1_001.fastq.gz,NaN,input-SiNipbl_S10_L004_R1_001.fastq.gz,hg19,mm9,,input_siNipbl_S10,,/storage/scratch01/users/dgimenezl/genomes/hum...,2864785220,/storage/scratch01/users/dgimenezl/genomes/mou...,/storage/scratch01/users/aquevedo/genomes/huma...,input-SiNipbl_S10_L004_R1_001,input_siNipbl,no
8,NiPBL,siC,S4,L001_R1_001.fastq.gz,NaN,Sic-NiPBL_S4_L001_R1_001.fastq.gz,hg19,mm9,input_siC_S9,NiPBL_siC_S4,input_siC,/storage/scratch01/users/dgimenezl/genomes/hum...,2864785220,/storage/scratch01/users/dgimenezl/genomes/mou...,/storage/scratch01/users/aquevedo/genomes/huma...,Sic-NiPBL_S4_L001_R1_001,NiPBL_siC,no
9,NiPBL,siC,S4,L002_R1_001.fastq.gz,NaN,Sic-NiPBL_S4_L002_R1_001.fastq.gz,hg19,mm9,input_siC_S9,NiPBL_siC_S4,input_siC,/storage/scratch01/users/dgimenezl/genomes/hum...,2864785220,/storage/scratch01/users/dgimenezl/genomes/mou...,/storage/scratch01/users/aquevedo/genomes/huma...,Sic-NiPBL_S4_L002_R1_001,NiPBL_siC,no


In [42]:
expand("bw/{Prot_Cond}_mean.bw" ,
            Prot_Cond=data.Prot_Cond[
            (data.Protein!="input")&
            (data.MergeBW =="yes")].unique())

['bw/SA2_siC_mean.bw', 'bw/SMC1_siC_mean.bw']

In [43]:
expand("bw/{Samp}_RPKM_scaled.bw",
         Samp=data.Samples[
                         (data.MergeBW == 'yes') &
                         (data.Protein == "SA2") & 
                         (data.Condition == "siC")].unique())

['bw/SA2_siC_S11_RPKM_scaled.bw', 'bw/SA2_siC_S2_RPKM_scaled.bw']

## Generate Arguments for MergeSamFiles
Demonstrate how the command string for MergeSamFiles is produced. We take advantage of the `data` table and the auxiliary function `Input_merge_bam`

In [44]:
def Input_merge_bam(*bams):
    '''
    Generate formated string for -I option for MergeSamFiles
    '''
    input_bams=["-I " + str(b) for b in bams]
    input_bams=" ".join(input_bams)
    return input_bams

In [45]:
## generate a list with .bam filenames the same way as in the Snakefile
sample=data.Samples[(data.Protein == 'SA2') & (data.Condition == 'siC')].unique()
print("sample: ", sample)
bams=[s +'_final.bam' for s in sample]
print("Input bams: ", bams)

sample:  ['SA2_siC_S11' 'SA2_siC_S2']
Input bams:  ['SA2_siC_S11_final.bam', 'SA2_siC_S2_final.bam']


In [46]:
## Apply function to the list of bams filenames
Input_merge_bam(*bams)

'-I SA2_siC_S11_final.bam -I SA2_siC_S2_final.bam'

In [47]:
expand(DATADIR+"/align/{Samp}_final.bam",
       Samp=data.Samples[(data.Protein == 'SMC1') & (data.Condition=='siC')].unique())

['/storage/scratch01/users/aquevedo/6Nov-siNipbl-newSnak/data//align/SMC1_siC_S12_final.bam',
 '/storage/scratch01/users/aquevedo/6Nov-siNipbl-newSnak/data//align/SMC1_siC_S3_final.bam']

In [48]:
## Esto es una prueba, quitarlo


" ".join(
    np.unique(
        expand({"/bw/{sample}_RPKM_scaled.bw"},
                       sample=list(filter(None,
                                          [ sampl \
                                    if prot != "input" \
                                    else "" \
         for (sampl,prot) in zip(data.Samples, data.Protein)]
                                 )
                          )
               )
    )
)


'/bw/NiPBL_siC_S4_RPKM_scaled.bw /bw/NiPBL_siNipbl_S8_RPKM_scaled.bw /bw/SA1_siC_S1_RPKM_scaled.bw /bw/SA1_siNipbl_S5_RPKM_scaled.bw /bw/SA2_siC_S11_RPKM_scaled.bw /bw/SA2_siC_S2_RPKM_scaled.bw /bw/SA2_siNipbl_S6_RPKM_scaled.bw /bw/SMC1_siC_S12_RPKM_scaled.bw /bw/SMC1_siC_S3_RPKM_scaled.bw /bw/SMC1_siNipbl_S7_RPKM_scaled.bw'

In [49]:
data.Samples.unique()

array(['input_siC_S9', 'input_siNipbl_S10', 'NiPBL_siC_S4', 'SA1_siC_S1',
       'SA2_siC_S11', 'SA2_siC_S2', 'SMC1_siC_S12', 'SMC1_siC_S3',
       'NiPBL_siNipbl_S8', 'SA1_siNipbl_S5', 'SA2_siNipbl_S6',
       'SMC1_siNipbl_S7'], dtype=object)

In [50]:
expand('{a}.gz', a=np.arange(0,10))

['0.gz',
 '1.gz',
 '2.gz',
 '3.gz',
 '4.gz',
 '5.gz',
 '6.gz',
 '7.gz',
 '8.gz',
 '9.gz']

In [51]:
data[23:25]

,Protein,Condition,Rep,Ext,Run,File,Genome,Norm,Input,Samples,InputMerged,PATH_genome,Genome_size,PATH_genome_cal,PATH_refSeq_genes,fqBasename,Prot_Cond,MergeBW
23,SA2,siC,S2,L004_R1_001.fastq.gz,NaN,Sic-SA2_S2_L004_R1_001.fastq.gz,hg19,mm9,input_siC_S9,SA2_siC_S2,input_siC,/storage/scratch01/users/dgimenezl/genomes/hum...,2864785220,/storage/scratch01/users/dgimenezl/genomes/mou...,/storage/scratch01/users/aquevedo/genomes/huma...,Sic-SA2_S2_L004_R1_001,SA2_siC,yes
24,SMC1,siC,S12,L001_R1_001.fastq.gz,NaN,Sic-SMC1-new_S12_L001_R1_001.fastq.gz,hg19,mm9,input_siC_S9,SMC1_siC_S12,input_siC,/storage/scratch01/users/dgimenezl/genomes/hum...,2864785220,/storage/scratch01/users/dgimenezl/genomes/mou...,/storage/scratch01/users/aquevedo/genomes/huma...,Sic-SMC1-new_S12_L001_R1_001,SMC1_siC,yes


In [66]:
## Create column to rule which libraries have more than 1 replicate and must be merged

def multipleReps(sample):
    prot, cond, _ = sample.split("_")
    reps = len(data.Rep[(data.Protein == prot) & (data.Condition == cond)].unique())
    
    return reps > 1

data["MergeReplicates"]=[multipleReps(sample) for sample in data.Samples]
    
    

In [68]:
data

,Protein,Condition,Rep,Ext,Run,File,Genome,Norm,Input,Samples,InputMerged,PATH_genome,Genome_size,PATH_genome_cal,PATH_refSeq_genes,fqBasename,Prot_Cond,MergeBW,MergeReplicates
0,input,siC,S9,L001_R1_001.fastq.gz,NaN,input-SiC_S9_L001_R1_001.fastq.gz,hg19,mm9,,input_siC_S9,,/storage/scratch01/users/dgimenezl/genomes/hum...,2864785220,/storage/scratch01/users/dgimenezl/genomes/mou...,/storage/scratch01/users/aquevedo/genomes/huma...,input-SiC_S9_L001_R1_001,input_siC,no,False
1,input,siC,S9,L002_R1_001.fastq.gz,NaN,input-SiC_S9_L002_R1_001.fastq.gz,hg19,mm9,,input_siC_S9,,/storage/scratch01/users/dgimenezl/genomes/hum...,2864785220,/storage/scratch01/users/dgimenezl/genomes/mou...,/storage/scratch01/users/aquevedo/genomes/huma...,input-SiC_S9_L002_R1_001,input_siC,no,False
2,input,siC,S9,L003_R1_001.fastq.gz,NaN,input-SiC_S9_L003_R1_001.fastq.gz,hg19,mm9,,input_siC_S9,,/storage/scratch01/users/dgimenezl/genomes/hum...,2864785220,/storage/scratch01/users/dgimenezl/genomes/mou...,/storage/scratch01/users/aquevedo/genomes/huma...,input-SiC_S9_L003_R1_001,input_siC,no,False
3,input,siC,S9,L004_R1_001.fastq.gz,NaN,input-SiC_S9_L004_R1_001.fastq.gz,hg19,mm9,,input_siC_S9,,/storage/scratch01/users/dgimenezl/genomes/hum...,2864785220,/storage/scratch01/users/dgimenezl/genomes/mou...,/storage/scratch01/users/aquevedo/genomes/huma...,input-SiC_S9_L004_R1_001,input_siC,no,False
4,input,siNipbl,S10,L001_R1_001.fastq.gz,NaN,input-SiNipbl_S10_L001_R1_001.fastq.gz,hg19,mm9,,input_siNipbl_S10,,/storage/scratch01/users/dgimenezl/genomes/hum...,2864785220,/storage/scratch01/users/dgimenezl/genomes/mou...,/storage/scratch01/users/aquevedo/genomes/huma...,input-SiNipbl_S10_L001_R1_001,input_siNipbl,no,False
5,input,siNipbl,S10,L002_R1_001.fastq.gz,NaN,input-SiNipbl_S10_L002_R1_001.fastq.gz,hg19,mm9,,input_siNipbl_S10,,/storage/scratch01/users/dgimenezl/genomes/hum...,2864785220,/storage/scratch01/users/dgimenezl/genomes/mou...,/storage/scratch01/users/aquevedo/genomes/huma...,input-SiNipbl_S10_L002_R1_001,input_siNipbl,no,False
6,input,siNipbl,S10,L003_R1_001.fastq.gz,NaN,input-SiNipbl_S10_L003_R1_001.fastq.gz,hg19,mm9,,input_siNipbl_S10,,/storage/scratch01/users/dgimenezl/genomes/hum...,2864785220,/storage/scratch01/users/dgimenezl/genomes/mou...,/storage/scratch01/users/aquevedo/genomes/huma...,input-SiNipbl_S10_L003_R1_001,input_siNipbl,no,False
7,input,siNipbl,S10,L004_R1_001.fastq.gz,NaN,input-SiNipbl_S10_L004_R1_001.fastq.gz,hg19,mm9,,input_siNipbl_S10,,/storage/scratch01/users/dgimenezl/genomes/hum...,2864785220,/storage/scratch01/users/dgimenezl/genomes/mou...,/storage/scratch01/users/aquevedo/genomes/huma...,input-SiNipbl_S10_L004_R1_001,input_siNipbl,no,False
8,NiPBL,siC,S4,L001_R1_001.fastq.gz,NaN,Sic-NiPBL_S4_L001_R1_001.fastq.gz,hg19,mm9,input_siC_S9,NiPBL_siC_S4,input_siC,/storage/scratch01/users/dgimenezl/genomes/hum...,2864785220,/storage/scratch01/users/dgimenezl/genomes/mou...,/storage/scratch01/users/aquevedo/genomes/huma...,Sic-NiPBL_S4_L001_R1_001,NiPBL_siC,no,False
9,NiPBL,siC,S4,L002_R1_001.fastq.gz,NaN,Sic-NiPBL_S4_L002_R1_001.fastq.gz,hg19,mm9,input_siC_S9,NiPBL_siC_S4,input_siC,/storage/scratch01/users/dgimenezl/genomes/hum...,2864785220,/storage/scratch01/users/dgimenezl/genomes/mou...,/storage/scratch01/users/aquevedo/genomes/huma...,Sic-NiPBL_S4_L002_R1_001,NiPBL_siC,no,False


## Filenames and Labels for compute_matrix

In [ ]:
matrixFiles = data.Samples[data.Protein != 'input'].unique()
matrixFiles

In [ ]:
import re

In [ ]:
[re.sub("_S.+$","",f) for f in matrixFiles]

## Find samples to cluster heatmap

In [ ]:
## Find where SA2 protein appears in matr
ix=np.where(np.array(["SA2" in Samp for Samp in matrixFiles ]))[0]
ix=ix+1 # first column is 1 in deeptools but np.where is 0 based
" ".join(ix.astype(str))

## Motif Enrichment

We are using homer's `findMotifsGenome.pl` script to find e riched motifs between conditions.
We must be able to 

In [ ]:
PEAKSDIR = "/Users/aqo/Desktop/siNipblChipPeaks/unique/"

In [ ]:
## Data frame with .narrowPeak files
naPeak = glob.glob(PEAKSDIR + "*uniquePeaks.bed")
meta=pd.DataFrame(naPeak, columns=["uniqPeakFile"])

In [ ]:
## Extract basename
meta["Basename"]=[re.sub(pattern='^.+/', repl="",string=f) for f in meta.uniqPeakFile]
meta


In [ ]:
## EXtract Prot Condition and Rep from Basename
meta[["Protein","uniqueIn","Treatment","tmp"]] = meta["Basename"].str.split("_", expand=True)
meta=meta.drop(columns=['tmp'])

In [ ]:
meta['Sample']=meta.Protein +"_"+meta.uniqueIn+"_"+ meta.Treatment
meta

In [ ]:
## add Background col to know which file use as bg in homer
meta['Background']= [ meta.uniqPeakFile[(meta.Protein == P) & (meta.uniqueIn == 'Common')].values[0] \
                     if (UniqIn != 'Common') else "" \
                     for P,UniqIn in zip(meta.Protein, meta.uniqueIn)]
meta

In [ ]:
meta.Background.astype(str)

## MOREY

In [2]:
data = pd.read_csv("../config/table_MoreyPaper_Polycomb_GSE107176.tsv", sep="\t")

In [21]:
## Genome bowtie2 index prefixes paths
genome_path = {
	"mm9":"/storage/scratch01/users/dgimenezl/genomes/mouse/mm9/mm9",
    "mm10":"/storage/scratch01/users/dgimenezl/genomes/mouse/mm10/mm10" ,
    "hg19":"/storage/scratch01/users/dgimenezl/genomes/human/hg19/hg19",
    "hg38":"/storage/scratch01/users/dgimenezl/genomes/human/hg38/hg38",
    "-":"NO_CALIBRATION"}
refSeq_genes_path = {
	"mm9" : "",
	"mm10" : "",
	"hg19" : "/storage/scratch01/users/aquevedo/genomes/human/hg19/hg19_RefSeqCuratedGenes.bed",
	"hg38" : ""
}
## Genome sizes for big wig computation
genome_size={"mm9":2620345972,
    "mm10":2652783500,
    "hg19":2864785220,
    "hg38":2913022398}

data["Samples"] = data.Protein +"_"+ data.Condition +"_"+ data.Rep
# Match each sample reads with appropriate input to calculate calibration factor
data["Input"] = [ data.Samples[(data.Protein=="input") & (data.Condition==Cond)].values[0] \
                 if Prot != "input" \
                 else "" \
                 for Prot,Cond in zip(data.Protein,data.Condition)  ]

# Input for calling peaks after merging replicates
data["InputMerged"] = [ re.sub("_[SR][0-9]+$","", ip) if ip != ""
                       else "" for ip in data.Input]

# All different Prot_Cond prosibilities to merge replicates
data["Prot_Cond"] = ["_".join((Prot,Cond)) for Prot,Cond in zip(data.Protein,data.Condition)]

data["PATH_genome"] = [genome_path[i] for i in data.Genome] 

data["Genome_size"] = [genome_size[i] for i in data.Genome]

data["PATH_genome_cal"] = [genome_path[i] for i in data.Norm]

data["PATH_refSeq_genes"] = [refSeq_genes_path[i] for i in data.Genome]
## Remove .fastq.gz to use basename with expand() in rule "all"
data["fqBasename"] = [f.replace(".fastq.gz","") for f in data["File"]]


In [30]:
data.PATH_genome_cal.values

array(['NO_CALIBRATION', 'NO_CALIBRATION', 'NO_CALIBRATION',
       'NO_CALIBRATION'], dtype=object)

In [34]:
calGenIx = data.PATH_genome_cal[data.Samples=="RING1B_control_R1"].values[0]

In [35]:
calGenIx

'NO_CALIBRATION'

In [37]:
isCalMesage = ">>> CALIBRATED <<<"
noCalMesage = ">>> NOT CALIBRATED <<<"
print("!!!!! :" + calGenIx)

if calGenIx == "NO_CALIBRATION": ## If NO calibration. 
    ## Check we are in this case by distinct bowtie flags using snakemake -p option
    print(f"{noCalMesage:^80}")
else: ## If YES calibration
	print(f"{isCalMesage:^80}")

!!!!! :NO_CALIBRATION
                             >>> NOT CALIBRATED <<<                             


## NIPBL in Neutrophils

In [2]:
data = pd.read_csv("../config/SraRunTable_NIPBL_calcium_Neutrophils_GSE154051.tsv", sep="\t")

In [2]:
## Genome bowtie2 index prefixes paths
genome_path = {
	"mm9":"/storage/scratch01/users/dgimenezl/genomes/mouse/mm9/mm9",
    "mm10":"/storage/scratch01/users/dgimenezl/genomes/mouse/mm10/mm10" ,
    "hg19":"/storage/scratch01/users/dgimenezl/genomes/human/hg19/hg19",
    "hg38":"/storage/scratch01/users/dgimenezl/genomes/human/hg38/hg38",
    "-":"NO_CALIBRATION"}
refSeq_genes_path = {
	"mm9" : "",
	"mm10" : "",
	"hg19" : "/storage/scratch01/users/aquevedo/genomes/human/hg19/hg19_RefSeqCuratedGenes.bed",
	"hg38" : ""
}
## Genome sizes for big wig computation
genome_size={"mm9":2620345972,
    "mm10":2652783500,
    "hg19":2864785220,
    "hg38":2913022398}


In [27]:
data["Samples"] = data.Protein +"_"+ data.Condition +"_"+ data.Rep
# Match each sample reads with appropriate input to calculate calibration factor
data["Input"] = [ data.Samples[(data.Protein=="input") & (data.Condition==Cond)].values[0] \
                 if Prot != "input" and Cond in data.Condition[data.Protein=="input"].values \
                 else "" \
                 for Prot,Cond in zip(data.Protein,data.Condition)  ]

In [28]:
data

,Run,Protein,Condition,Rep,Ext,File,Genome,Norm,Treatment,chip_antibody,...,Cell_type,Experiment,Genotype,LibraryLayout,Organism,source_name,SRA Study,STRAIN,Samples,Input
0,SRR12179324,Nipbl,UT,R1,.fastq.gz,Nipbl_UT_R1.fastq.gz,mm9,-,untreated,NIPBL A301-779A Bethyl Laboratries,...,ECOMG derived neutrophils,SRX8693934,"human E2A_ER_Pbx1b\, wild type",SINGLE,Mus musculus,in-vitro differentiated neutrophils,SRP270993,C57BL/6,Nipbl_UT_R1,
1,SRR12179325,Nipbl,Fast,R1,.fastq.gz,Nipbl_Fast_R1.fastq.gz,mm9,-,fast_treated with 20uM A23187 for 15min,NIPBL A301-779A Bethyl Laboratries,...,ECOMG derived neutrophils,SRX8693935,"human E2A_ER_Pbx1b\, wild type",SINGLE,Mus musculus,in-vitro differentiated neutrophils,SRP270993,C57BL/6,Nipbl_Fast_R1,input_Fast_R1
2,SRR12179327,Nipbl,UT,R2,.fastq.gz,Nipbl_UT_R2.fastq.gz,mm9,-,untreated,NIPBL A301-779A Bethyl Laboratries,...,ECOMG derived neutrophils,SRX8693937,"human E2A_ER_Pbx1b\, wild type",SINGLE,Mus musculus,in-vitro differentiated neutrophils,SRP270993,C57BL/6,Nipbl_UT_R2,
3,SRR12179328,Nipbl,Fast,R2,.fastq.gz,Nipbl_Fast_R2.fastq.gz,mm9,-,fast_treated with 20uM A23187 for 15min,NIPBL A301-779A Bethyl Laboratries,...,ECOMG derived neutrophils,SRX8693938,"human E2A_ER_Pbx1b\, wild type",SINGLE,Mus musculus,in-vitro differentiated neutrophils,SRP270993,C57BL/6,Nipbl_Fast_R2,input_Fast_R1
4,SRR12179330,Nipbl,UT,R3,.fastq.gz,Nipbl_UT_R3.fastq.gz,mm9,-,untreated,NIPBL A301-779A Bethyl Laboratries,...,ECOMG derived neutrophils,SRX8693940,"human E2A_ER_Pbx1b\, wild type",SINGLE,Mus musculus,in-vitro differentiated neutrophils,SRP270993,C57BL/6,Nipbl_UT_R3,
5,SRR12179331,Nipbl,Fast,R3,.fastq.gz,Nipbl_Fast_R3.fastq.gz,mm9,-,fast_treated with 20uM A23187 for 15min,NIPBL A301-779A Bethyl Laboratries,...,ECOMG derived neutrophils,SRX8693941,"human E2A_ER_Pbx1b\, wild type",SINGLE,Mus musculus,in-vitro differentiated neutrophils,SRP270993,C57BL/6,Nipbl_Fast_R3,input_Fast_R1
6,SRR12179333,Nipbl,UT,R4,.fastq.gz,Nipbl_UT_R4.fastq.gz,mm9,-,untreated,NIPBL A301-779A Bethyl Laboratries,...,ECOMG derived neutrophils,SRX8693943,"human E2A_ER_Pbx1b\, wild type",SINGLE,Mus musculus,in-vitro differentiated neutrophils,SRP270993,C57BL/6,Nipbl_UT_R4,
7,SRR12179334,Nipbl,Fast,R4,.fastq.gz,Nipbl_Fast_R4.fastq.gz,mm9,-,fast_treated with 20uM A23187 for 15min,NIPBL A301-779A Bethyl Laboratries,...,ECOMG derived neutrophils,SRX8693944,"human E2A_ER_Pbx1b\, wild type",SINGLE,Mus musculus,in-vitro differentiated neutrophils,SRP270993,C57BL/6,Nipbl_Fast_R4,input_Fast_R1
8,SRR12179336,Smc1,UT,R1,.fastq.gz,Smc1_UT_R1.fastq.gz,mm9,-,untreated,SMC1 A300-055A Bethyl Laboratries,...,ECOMG derived neutrophils,SRX8693946,"human E2A_ER_Pbx1b\, wild type",SINGLE,Mus musculus,in-vitro differentiated neutrophils,SRP270993,C57BL/6,Smc1_UT_R1,
9,SRR12179337,Smc1,Fast,R1,.fastq.gz,Smc1_Fast_R1.fastq.gz,mm9,-,fast_treated with 20uM A23187 for 15min,SMC1 A300-055A Bethyl Laboratries,...,ECOMG derived neutrophils,SRX8693947,"human E2A_ER_Pbx1b\, wild type",SINGLE,Mus musculus,in-vitro differentiated neutrophils,SRP270993,C57BL/6,Smc1_Fast_R1,input_Fast_R1


In [12]:
Cond = "Fast"
Cond in data.Condition[data.Protein!="input"].values

True

In [17]:
type(data.Condition[data.Protein!="input"].values)

numpy.ndarray

# Ewing Cells

In [5]:
data = pd.read_csv("../config/table_ewing_WT_SA2KO_chip_metadata.tsv", sep="\t")

## Genome bowtie2 index prefixes paths
genome_path = {
	"mm9":"/storage/scratch01/users/dgimenezl/genomes/mouse/mm9/mm9",
    "mm10":"/storage/scratch01/users/dgimenezl/genomes/mouse/mm10/mm10" ,
    "hg19":"/storage/scratch01/users/dgimenezl/genomes/human/hg19/hg19",
    "hg38":"/storage/scratch01/users/dgimenezl/genomes/human/hg38/hg38",
    "-":"NO_CALIBRATION"}
refSeq_genes_path = {
	"mm9" : "",
	"mm10" : "",
	"hg19" : "/storage/scratch01/users/aquevedo/genomes/human/hg19/hg19_RefSeqCuratedGenes.bed",
	"hg38" : ""
}
## Genome sizes for big wig computation
genome_size={"mm9":2620345972,
    "mm10":2652783500,
    "hg19":2864785220,
    "hg38":2913022398}

data["Samples"] = data.Protein +"_"+ data.Condition +"_"+ data.Rep
# Match each sample reads with appropriate input to calculate calibration factor
data["Input"] = [ data.Samples[(data.Protein=="input") & (data.Condition==Cond)].values[0] \
                 if Prot != "input" and Cond in data.Condition[data.Protein=="input"].values \
                 else "" \
                 for Prot,Cond in zip(data.Protein,data.Condition)  ]

# Input for calling peaks after merging replicates
data["InputMerged"] = [ re.sub("_[SR][0-9]+$","", ip) if ip != ""
                       else "" for ip in data.Input]

# All different Prot_Cond prosibilities to merge replicates
data["Prot_Cond"] = ["_".join((Prot,Cond)) for Prot,Cond in zip(data.Protein,data.Condition)]

data["PATH_genome"] = [genome_path[i] for i in data.Genome] 

data["Genome_size"] = [genome_size[i] for i in data.Genome]

data["PATH_genome_cal"] = [genome_path[i] for i in data.Norm]

data["PATH_refSeq_genes"] = [refSeq_genes_path[i] for i in data.Genome]
## Remove .fastq.gz to use basename with expand() in rule "all"
data["fqBasename"] = [f.replace(".fastq.gz","") for f in data["File"]]



In [6]:
data

,Cell_Line,Protein,Condition,Rep,File,Genome,Norm,Comments,Samples,Input,InputMerged,Prot_Cond,PATH_genome,Genome_size,PATH_genome_cal,PATH_refSeq_genes,fqBasename
0,A637,FLI1,5.A-SA2KO,R1,5-FLI1_S19_L001_R1_001.fastq.gz,hg19,mm9,NaN,FLI1_5.A-SA2KO_R1,input_5.A-SA2KO_R1,input_5.A-SA2KO,FLI1_5.A-SA2KO,/storage/scratch01/users/dgimenezl/genomes/hum...,2864785220,/storage/scratch01/users/dgimenezl/genomes/mou...,/storage/scratch01/users/aquevedo/genomes/huma...,5-FLI1_S19_L001_R1_001
1,A637,FLI1,5.A-SA2KO,R1,5-FLI1_S19_L002_R1_001.fastq.gz,hg19,mm9,NaN,FLI1_5.A-SA2KO_R1,input_5.A-SA2KO_R1,input_5.A-SA2KO,FLI1_5.A-SA2KO,/storage/scratch01/users/dgimenezl/genomes/hum...,2864785220,/storage/scratch01/users/dgimenezl/genomes/mou...,/storage/scratch01/users/aquevedo/genomes/huma...,5-FLI1_S19_L002_R1_001
2,A637,FLI1,5.A-SA2KO,R1,5-FLI1_S19_L003_R1_001.fastq.gz,hg19,mm9,NaN,FLI1_5.A-SA2KO_R1,input_5.A-SA2KO_R1,input_5.A-SA2KO,FLI1_5.A-SA2KO,/storage/scratch01/users/dgimenezl/genomes/hum...,2864785220,/storage/scratch01/users/dgimenezl/genomes/mou...,/storage/scratch01/users/aquevedo/genomes/huma...,5-FLI1_S19_L003_R1_001
3,A637,FLI1,5.A-SA2KO,R1,5-FLI1_S19_L004_R1_001.fastq.gz,hg19,mm9,NaN,FLI1_5.A-SA2KO_R1,input_5.A-SA2KO_R1,input_5.A-SA2KO,FLI1_5.A-SA2KO,/storage/scratch01/users/dgimenezl/genomes/hum...,2864785220,/storage/scratch01/users/dgimenezl/genomes/mou...,/storage/scratch01/users/aquevedo/genomes/huma...,5-FLI1_S19_L004_R1_001
4,A637,R1B,5.A-SA2KO,R1,5-R1B_S16_L001_R1_001.fastq.gz,hg19,mm9,NaN,R1B_5.A-SA2KO_R1,input_5.A-SA2KO_R1,input_5.A-SA2KO,R1B_5.A-SA2KO,/storage/scratch01/users/dgimenezl/genomes/hum...,2864785220,/storage/scratch01/users/dgimenezl/genomes/mou...,/storage/scratch01/users/aquevedo/genomes/huma...,5-R1B_S16_L001_R1_001
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
75,A637,H3me3,2-WT,R1,2-H3me3_S13_L004_R1_001.fastq.gz,hg19,mm9,NaN,H3me3_2-WT_R1,input_2-WT_R1,input_2-WT,H3me3_2-WT,/storage/scratch01/users/dgimenezl/genomes/hum...,2864785220,/storage/scratch01/users/dgimenezl/genomes/mou...,/storage/scratch01/users/aquevedo/genomes/huma...,2-H3me3_S13_L004_R1_001
76,A637,input,2-WT,R1,INPUT-2-NEW_S4_L001_R1_001.fastq.gz,hg19,mm9,NaN,input_2-WT_R1,,,input_2-WT,/storage/scratch01/users/dgimenezl/genomes/hum...,2864785220,/storage/scratch01/users/dgimenezl/genomes/mou...,/storage/scratch01/users/aquevedo/genomes/huma...,INPUT-2-NEW_S4_L001_R1_001
77,A637,input,2-WT,R1,INPUT-2-NEW_S4_L002_R1_001.fastq.gz,hg19,mm9,NaN,input_2-WT_R1,,,input_2-WT,/storage/scratch01/users/dgimenezl/genomes/hum...,2864785220,/storage/scratch01/users/dgimenezl/genomes/mou...,/storage/scratch01/users/aquevedo/genomes/huma...,INPUT-2-NEW_S4_L002_R1_001
78,A637,input,2-WT,R1,INPUT-2-NEW_S4_L003_R1_001.fastq.gz,hg19,mm9,NaN,input_2-WT_R1,,,input_2-WT,/storage/scratch01/users/dgimenezl/genomes/hum...,2864785220,/storage/scratch01/users/dgimenezl/genomes/mou...,/storage/scratch01/users/aquevedo/genomes/huma...,INPUT-2-NEW_S4_L003_R1_001


In [8]:
expand("bw/{sample}_RPKM_scaled.bw",
				sample=data.Samples[data.PATH_genome_cal!=""].unique())

['bw/FLI1_5.A-SA2KO_R1_RPKM_scaled.bw',
 'bw/R1B_5.A-SA2KO_R1_RPKM_scaled.bw',
 'bw/input_5.A-SA2KO_R1_RPKM_scaled.bw',
 'bw/H3AC_5-SA2KO_R1_RPKM_scaled.bw',
 'bw/H3me3_5-SA2KO_R1_RPKM_scaled.bw',
 'bw/SMC1_5-SA2KO_R1_RPKM_scaled.bw',
 'bw/input_5-SA2KO_R1_RPKM_scaled.bw',
 'bw/FLI1_24.2-SA2KO_R1_RPKM_scaled.bw',
 'bw/H3AC_24.2-SA2KO_R1_RPKM_scaled.bw',
 'bw/R1B_24.2-SA2KO_R1_RPKM_scaled.bw',
 'bw/SMC1_24.2-SA2KO_R1_RPKM_scaled.bw',
 'bw/input_24.2-SA2KO_R1_RPKM_scaled.bw',
 'bw/SMC1_2.OLD-WT_R1_RPKM_scaled.bw',
 'bw/input_2.OLD-WT_R1_RPKM_scaled.bw',
 'bw/FLI1_2.C-WT_R1_RPKM_scaled.bw',
 'bw/R1B_2.C-WT_R1_RPKM_scaled.bw',
 'bw/input_2.C-WT_R1_RPKM_scaled.bw',
 'bw/H3AC_2-WT_R1_RPKM_scaled.bw',
 'bw/H3me3_2-WT_R1_RPKM_scaled.bw',
 'bw/input_2-WT_R1_RPKM_scaled.bw']